In [49]:
# Reference: https://pytorch.org/hub/ultralytics_yolov5/
import torch
import os
import cv2
import numpy as np

In [50]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in C:\Users\uttej/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-10-15 Python-3.11.3 torch-2.1.0+cu118 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [51]:
test_dir = 'D:\\present\\cmpe249-hw1\\dataset\\images\\test'
images = os.listdir(test_dir)

image_paths = []
for img_name in images:
	image_paths.append(test_dir + '\\' + img_name)

images = [cv2.imread(img_path) for img_path in image_paths]
images = [cv2.cvtColor(img, cv2.COLOR_BGR2RGB) for img in images]

In [52]:
# Get predictions
results = model(images)

In [75]:
# Store predictions in list of list of lists
df_lists = results.pandas().xyxy

all_detections = []
for df in df_lists:
	all_detections.append(df.values.tolist())

In [80]:
# Only select those with required categories
# Convert keys to lowercase to match with detections' categories
categories = {
    "pedestrian": 0,
    "truck": 1,
    "car": 2,
    "cyclist": 3,
    "dontCare": 4,
    "misc": 5,
    "van": 6,
    "tram": 7,
    "person_sitting": 8,
}

In [81]:
image_id = 0
coco_results = []
for image_detections in all_detections:
	image_id += 1
	for detection in image_detections:
		xmin, ymin, xmax, ymax, conf, clss, category = detection
		if category in categories:
			coco_result = {
				"image_id": image_id,
				"category_id": categories[category],
				"bbox": [xmin, ymin, xmax - xmin, ymax - ymin],
				"score": conf,
			}
			coco_results.append(coco_result)

In [83]:
import json

with open('predictions.json', 'w') as f:
	json.dump(coco_results, f)